# data load

In [61]:
import numpy as np
import datetime as dt
import pandas as pd
import math as mt
%matplotlib inline

from dateutil.relativedelta import relativedelta

dir = "C:/Users/ruich/\Desktop/New folder/lease/"
order_book = pd.read_excel(dir + "2024.01_2/平台_财务订单明细构成_- (基础台数据)(1)(1)(2).xlsx",
                          sheet_name = 0,
                          usecols="B:V")
order_book = order_book.fillna(0)
col_names = ["order", "name", "seller", "order_time", "device", "pur_px", "official_px"] + \
            ["term" + str(i) for i in range(1,13)] + \
            ["deposit", "final_payment"]

order_book.columns = col_names
order_book["order"] = order_book["order"].str.strip()

order_price_9_10 = pd.read_excel(dir + "2024.01/平台_财务订单明细构成_- (基础台数据).xlsx",
                          sheet_name = 0,
                          usecols="A:AO")

order_price_9_10 = order_price_9_10[["订单号","实付押金","买断金"]]
order_price_9_10.columns = ["order","deposit_fill","final_payment_fill"]
order_price_9_10["order"] = order_price_9_10["order"].str.strip()

order_book = order_book.merge(order_price_9_10, on = "order", how = "left")

order_book[["term1", "final_payment"]] = order_book[["term1", "final_payment"]].apply(pd.to_numeric, errors='coerce', axis=1)



payment_date = pd.read_excel(dir + "2024.01_2/华尔街-现金流量表.xlsx",
                          sheet_name = 0)
payment_date = payment_date.sort_values(["订单号", "还款时间"]).groupby("订单号").first() 
payment_date.reset_index(inplace=True)

payment_date = payment_date[["订单号", "还款时间"]]
payment_date.columns= ["order", "payment_date"]
payment_date["order"] = payment_date["order"].str.strip()

order_book = order_book.merge(payment_date, on = "order", how = "left")

# order_book.tail()

# order_book.to_excel(dir+"output.xlsx")

In [62]:
pur_data = pd.read_excel(dir + "2024.01/1期华尔街-商家投保客户.xlsx",
                          sheet_name = 0)
pur_data2 = pd.read_excel(dir + "2024.01/2期华尔街投保客户.xlsx",
                          sheet_name = 0)
pur_data = pd.concat([pur_data, pur_data2], ignore_index = True)

pur_data3 = pd.read_excel(dir + "2024.01/汇总.xlsx",
                          sheet_name = 1,
                          skiprows=2)
pur_data3 = pur_data3.iloc[:, [1, 6]]
pur_data3.columns = ["order", "pur_px_fill"]

# pur_data3[["pur_px_fill"]] = pur_data3[["pur_px_fill"]].apply(pd.to_numeric, errors='coerce', axis=1)

pur_data = pur_data[["订单编号","保额"]]
pur_data.columns = ["order", "pur_px_fill"]
pur_data = pd.concat([pur_data, pur_data3], ignore_index = True)
pur_data["order"] = pur_data["order"].str.strip()

order_book = order_book.merge(pur_data, on = "order", how = "left")

for col_i in ["deposit", "final_payment", "pur_px"]:
    order_book[col_i] = order_book[[col_i, col_i+"_fill"]].max(axis = 1)

In [63]:

order_book['order_time'] = pd.to_datetime(order_book['order_time'])
order_book['payment_date'] = np.where(order_book['payment_date'].isnull(), order_book['order_time'] + dt.timedelta(days=2), order_book['payment_date'])
order_book['pur_px'] = np.where(order_book['pur_px'] == 0, order_book['official_px'] - 700, order_book['pur_px'])
order_book = order_book[(order_book["order"] != "0") & (order_book["order"].notnull())]
order_book['lease_term'] = np.where(order_book['term12'] == 0, 6, 12)
order_book['term6'] = np.where(order_book['lease_term'] == 6, order_book['term6'] + order_book['final_payment'], order_book['term6'])
order_book['term12'] = np.where(order_book['lease_term'] == 12, order_book['term12'] + order_book['final_payment'], order_book['term12'])
order_book = order_book.drop(columns = [i + "_fill" for i in ["deposit", "final_payment", "pur_px"]]) \
                       .drop(columns = ["final_payment","seller"])
order_book["order_time"] = order_book["order_time"].dt.normalize()
order_book["payment_date"] = order_book["payment_date"].dt.normalize()

order_book["screen_insurance"] = np.maximum(order_book["term1"] - order_book["term2"], 0)
order_book["screen_insurance"] = np.where(order_book["screen_insurance"] <= 50, 0, order_book["screen_insurance"])
# order_book.to_excel(dir+"output.xlsx", index = False)

In [64]:
cashflow_book = order_book.melt(id_vars=["order","name","order_time","device","pur_px","official_px","deposit","lease_term","payment_date" ,"screen_insurance"])
cashflow_book["payment_date_new"] = cashflow_book["payment_date"] + (cashflow_book["variable"].str[4:].astype(int)-1).apply(lambda m: pd.DateOffset(months=m))
cashflow_book["payment_date_new"] = pd.to_datetime(cashflow_book["payment_date_new"]).dt.normalize()
cashflow_book["final_payment_date"] = cashflow_book["payment_date"] + (cashflow_book["lease_term"]-1).apply(lambda m: pd.DateOffset(months=m))
cashflow_book["final_payment_date"] = pd.to_datetime(cashflow_book["final_payment_date"]).dt.normalize()

C:\Users\ruich\AppData\Local\Temp\ipykernel_2568\2067689602.py:2: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  cashflow_book["payment_date_new"] = cashflow_book["payment_date"] + (cashflow_book["variable"].str[4:].astype(int)-1).apply(lambda m: pd.DateOffset(months=m))
C:\Users\ruich\AppData\Local\Temp\ipykernel_2568\2067689602.py:4: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  cashflow_book["final_payment_date"] = cashflow_book["payment_date"] + (cashflow_book["lease_term"]-1).apply(lambda m: pd.DateOffset(months=m))


In [65]:
default_data = pd.read_excel(dir + "2024.01_2/商家_逾期订单统计2024-02-03.xlsx",
                          sheet_name = 0)

# key ratios

In [66]:
as_of_dt = dt.datetime(2024, 1, 29)

initial_capital = 6500000
cash_balance = 300000
cash_balance_floor = 50000
cashflow_book = cashflow_book[cashflow_book["final_payment_date"]> as_of_dt] 
gross_receivable = cashflow_book[cashflow_book["payment_date_new"]> as_of_dt][["value"]].sum()
deposit = cashflow_book.groupby("order").agg(average = ("deposit","mean")).sum()

gross_ret = (gross_receivable[0] + cash_balance - deposit[0]) / initial_capital - 1
num_default = len(default_data.index)
default_rate = num_default / len(cashflow_book.index)

In [67]:
gross_receivable[0] + cash_balance

9335144.02

In [68]:
def order_rundown(cash_on_hand,
                  matrix_cash_left,
                  matrix):
    if matrix_cash_left > matrix["cash_cutoff_cumu"].max() + 0.0001:
        raise Exception("matrix_cash_left is more than total cash amount")
    current_bucket = matrix.index[matrix["cash_cutoff_rev_cumu"] < matrix_cash_left]
    current_bucket = matrix.index.max() if current_bucket.empty else current_bucket[0] - 1

    new_phones = 1
    orders = pd.DataFrame(columns = ["device","onedollar","lease_term","deposit_mean","pur_px_mean","screen_ins_mean",
                                     "official_px_mean","monthly_pay_mean","px_multiplier_mean","num_device","cost"])
    counter = 0
    cash_on_hand_beginning = cash_on_hand
    while (cash_on_hand > matrix.loc[current_bucket]["pur_px_mean"]) & (counter < 500):

        bucket_cash_left = matrix.loc[current_bucket]["cash_cutoff"] - (matrix.loc[current_bucket]["cash_cutoff_rev_cumu"] - matrix_cash_left)
        pre_bucket_cash_left = 0 if counter == 0 else matrix_cash_left - matrix.loc[current_bucket]["cash_cutoff_rev_cumu"] 

        total_bucket_cash = bucket_cash_left + pre_bucket_cash_left
        new_phones = mt.floor(min(cash_on_hand, total_bucket_cash) / matrix.loc[current_bucket]["pur_px_mean"])
        new_phones_cost = new_phones * matrix.loc[current_bucket]["pur_px_mean"]
        orders = pd.concat([orders, 
                            pd.DataFrame({"device": [matrix.loc[current_bucket]["device"]],
                                          "onedollar": [matrix.loc[current_bucket]["onedollar"]],
                                          "lease_term": [matrix.loc[current_bucket]["lease_term"]],
                                          "deposit_mean": [matrix.loc[current_bucket]["deposit_mean"]],
                                          "pur_px_mean": [matrix.loc[current_bucket]["pur_px_mean"]],
                                          "screen_ins_mean": [matrix.loc[current_bucket]["screen_ins_mean"]],
                                          "official_px_mean": [matrix.loc[current_bucket]["official_px_mean"]],
                                          "monthly_pay_mean": [matrix.loc[current_bucket]["monthly_pay_mean"]],
                                          "px_multiplier_mean": [matrix.loc[current_bucket]["px_multiplier_mean"]],
                                          "num_device": [new_phones],
                                          "cost": [new_phones_cost]})])
        orders = orders[orders["num_device"] != 0]
        cash_on_hand = cash_on_hand - new_phones_cost
        counter = counter + 1
        if current_bucket == matrix.index.max(): 
            current_bucket = 1
            matrix_cash_left = matrix["cash_cutoff_cumu"].max()
        else:
            current_bucket = current_bucket + 1
            matrix_cash_left = matrix_cash_left - new_phones_cost
    if counter == 500:
        raise Exception("infinite while loop")
    
    cash_on_hand_left = cash_on_hand_beginning - orders["cost"].sum()

    return orders, matrix_cash_left, cash_on_hand_left

def order_allocate(new_orders,
                   delayed_orders,
                   daily_cap = 30):
    # new_orders = orders_i
    # delayed_orders = pd.DataFrame()
    # daily_cap  = 30
    if daily_cap >= new_orders["num_device"].sum():
        alloc_orders = new_orders.copy()
        unalloc_orders = pd.DataFrame()
    else:
        new_orders.index = range(1,len(new_orders.index) + 1)
        new_orders["num_device_cumu"] = new_orders["num_device"].cumsum()

        current_bucket = new_orders.index[new_orders["num_device_cumu"] >= daily_cap]
        current_bucket = current_bucket[0]

        alloc_orders = new_orders.loc[1:current_bucket].copy()
        alloc_orders.loc[current_bucket,"num_device"] = new_orders.loc[current_bucket]["num_device_cumu"] - daily_cap if current_bucket != 1 else daily_cap

        unalloc_orders =  new_orders.loc[current_bucket:new_orders.index.max()].copy()
        unalloc_orders.loc[current_bucket,"num_device"] = new_orders.loc[current_bucket]["num_device"] - \
                                                          alloc_orders.loc[current_bucket,"num_device"]
        alloc_orders = alloc_orders.drop("num_device_cumu", axis = 1)
        unalloc_orders = unalloc_orders.drop("num_device_cumu", axis = 1)

    delayed_orders = pd.concat([delayed_orders, unalloc_orders])

    return alloc_orders, delayed_orders

# reinvestment

In [69]:
# basic set up
reinvest_petiod        = 12 # month
traffic_fee_ratio      = 0.08 * 0.7
mayi_fee               = 0
insurance_fee_ratio    = 0.01
withdraw_fee_ratio     = 0
profit_share_fee       = 0.02
daily_volume_cap       = 50
neglect_device_cutoff  = 5

# baseline matrix
baseline_lookback = 90 # days
baseline_bop = as_of_dt - dt.timedelta(days=baseline_lookback)
baseline_hist = order_book[order_book["payment_date"] >= baseline_bop]
baseline_hist["device"] = baseline_hist["device"].str.replace('\W', '', regex = True).str.replace('[^\x00-\x7F]+', '', regex = True)
baseline_hist = baseline_hist[~baseline_hist["device"].str.contains("Pods|Pad|Watch|Book")]
baseline_hist["onedollar"] = np.where(baseline_hist["term1"] == 1, 1, 0)
deposit_avg = baseline_hist["deposit"].mean()
one_dollar_matrix0 = baseline_hist.groupby("onedollar").size()
one_dollar_matrix0 = one_dollar_matrix0 / one_dollar_matrix0.sum()
one_dollar_matrix = {}
one_dollar_matrix["percentage"] = one_dollar_matrix0.to_dict()

product_matrix = baseline_hist.groupby("device").agg({"pur_px":["mean", "count"]})
product_matrix.columns = product_matrix.columns.droplevel(0)
product_matrix["percentage"] = product_matrix["count"] / product_matrix["count"].sum()

lease_term_matrix0 = baseline_hist.groupby("lease_term").size()
lease_term_matrix0 = lease_term_matrix0 / lease_term_matrix0.sum()
lease_term_matrix = {}
lease_term_matrix["percentage"] = lease_term_matrix0.to_dict()
var_term = ["term" + str(i) for i in range(1,13)]
baseline_hist["px_multiplier"] = baseline_hist[var_term].sum(axis = 1) / baseline_hist["official_px"]
device_matrix = baseline_hist.groupby(["device", "onedollar", "lease_term"], as_index = False) \
                             .agg(deposit_mean = ("deposit", "mean"),
                                  pur_px_mean  = ("pur_px", "mean"),
                                  official_px_mean = ("official_px", "mean"),
                                  monthly_pay_mean = ("term2", "mean"),
                                  px_multiplier_mean = ("px_multiplier", "mean"),
                                  screen_ins_mean = ("screen_insurance", "mean"),
                                  order_count = ("order", "count")
                                 )

device_matrix = device_matrix[device_matrix["order_count"] >= neglect_device_cutoff]
## test
# device_matrix = device_matrix[device_matrix["device"]=="Mate60Pro"]

device_matrix["order_perc"] = device_matrix["order_count"] / device_matrix["order_count"].sum()
device_matrix = device_matrix.sort_values(by = ["order_perc"], ascending = False)
device_matrix["order_cutoff"] = round(device_matrix["order_count"] / device_matrix["order_count"].iloc[-1])
device_matrix["cash_cutoff"] = device_matrix["order_cutoff"] * device_matrix["pur_px_mean"]
device_matrix["cash_cutoff_cumu"] = device_matrix["cash_cutoff"].cumsum()
device_matrix["cash_cutoff_rev_cumu"] = device_matrix.cash_cutoff[::-1].cumsum()
device_matrix.index = range(1,len(device_matrix.index) + 1)

C:\Users\ruich\AppData\Local\Temp\ipykernel_2568\3093412389.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_hist["device"] = baseline_hist["device"].str.replace('\W', '', regex = True).str.replace('[^\x00-\x7F]+', '', regex = True)


In [70]:
device_matrix # profit margin

,device,onedollar,lease_term,deposit_mean,pur_px_mean,official_px_mean,monthly_pay_mean,px_multiplier_mean,screen_ins_mean,order_count,order_perc,order_cutoff,cash_cutoff,cash_cutoff_cumu,cash_cutoff_rev_cumu
1,iPhone15ProMax,0,12,524.197531,9049.846914,10008.876543,933.433679,1.277797,0.0,405,0.567227,58.0,524891.120988,524891.120988,899513.174978
2,iPhone15ProMax,1,12,374.545455,8908.409091,9999.000000,994.851818,1.290000,0.0,110,0.154062,16.0,142534.545455,667425.666442,374622.053991
3,iPhone15Pro,0,12,317.073171,8062.865854,8889.243902,892.049146,1.293419,0.0,82,0.114846,12.0,96754.390244,764180.056686,232087.508536
4,Mate60Pro,0,12,330.000000,7565.266667,7199.000000,476.449333,1.205889,0.0,30,0.042017,4.0,30261.066667,794441.123353,135333.118292
5,iPhone15Pro,1,12,268.965517,8128.758621,9033.482759,855.989310,1.271278,0.0,29,0.040616,4.0,32515.034483,826956.157836,105072.051626
6,iPhone15ProMax,0,6,1024.000000,9157.240000,9999.000000,1066.271600,1.236360,0.0,25,0.035014,4.0,36628.960000,863585.117836,72557.017143
7,iPhone15,0,12,453.303333,5659.600000,6332.333333,590.271333,1.275752,0.0,15,0.021008,2.0,11319.200000,874904.317836,35928.057143
8,Phone15Pro,0,12,1018.181818,8398.000000,8908.090909,874.943636,1.140229,0.0,11,0.015406,2.0,16796.000000,891700.317836,24608.857143
9,Mate60Pro,1,12,142.857143,7812.857143,7284.714286,523.088571,1.280004,0.0,7,0.009804,1.0,7812.857143,899513.174978,7812.857143


In [71]:
# matrix.loc[1]["cash_cutoff_rev_cumu"]

In [72]:
device_matrix

,device,onedollar,lease_term,deposit_mean,pur_px_mean,official_px_mean,monthly_pay_mean,px_multiplier_mean,screen_ins_mean,order_count,order_perc,order_cutoff,cash_cutoff,cash_cutoff_cumu,cash_cutoff_rev_cumu
1,iPhone15ProMax,0,12,524.197531,9049.846914,10008.876543,933.433679,1.277797,0.0,405,0.567227,58.0,524891.120988,524891.120988,899513.174978
2,iPhone15ProMax,1,12,374.545455,8908.409091,9999.000000,994.851818,1.290000,0.0,110,0.154062,16.0,142534.545455,667425.666442,374622.053991
3,iPhone15Pro,0,12,317.073171,8062.865854,8889.243902,892.049146,1.293419,0.0,82,0.114846,12.0,96754.390244,764180.056686,232087.508536
4,Mate60Pro,0,12,330.000000,7565.266667,7199.000000,476.449333,1.205889,0.0,30,0.042017,4.0,30261.066667,794441.123353,135333.118292
5,iPhone15Pro,1,12,268.965517,8128.758621,9033.482759,855.989310,1.271278,0.0,29,0.040616,4.0,32515.034483,826956.157836,105072.051626
6,iPhone15ProMax,0,6,1024.000000,9157.240000,9999.000000,1066.271600,1.236360,0.0,25,0.035014,4.0,36628.960000,863585.117836,72557.017143
7,iPhone15,0,12,453.303333,5659.600000,6332.333333,590.271333,1.275752,0.0,15,0.021008,2.0,11319.200000,874904.317836,35928.057143
8,Phone15Pro,0,12,1018.181818,8398.000000,8908.090909,874.943636,1.140229,0.0,11,0.015406,2.0,16796.000000,891700.317836,24608.857143
9,Mate60Pro,1,12,142.857143,7812.857143,7284.714286,523.088571,1.280004,0.0,7,0.009804,1.0,7812.857143,899513.174978,7812.857143


In [73]:

cashflow_fcst = pd.DataFrame()
cashflow_fcst["date"] = pd.date_range(as_of_dt,
                                      as_of_dt + relativedelta(months = reinvest_petiod + 11))
cashflow_known = cashflow_book.groupby("payment_date_new", as_index = False) \
                              .agg(netflow_old = ("value", "sum"))
cashflow_fcst = pd.merge(cashflow_fcst, cashflow_known, left_on = "date", right_on = "payment_date_new", how = "left").drop(["payment_date_new"], axis = 1)

new_cols = ["inflow_deposit", "inflow_screen_ins", "inflow_payment", 
              "outflow_phone_cost", "outflow_traffic_fee", "outflow_mayi_fee", 
              "outflow_insurance","netflow_new","orders_new", "cash_on_book"]
cashflow_fcst = pd.concat([cashflow_fcst, pd.DataFrame(columns=new_cols)])
cashflow_fcst.at[0, "cash_on_book"] = cash_balance + cashflow_fcst.at[0, "netflow_old"]

order_book_fcst = pd.DataFrame()
cashflow_book_fcst = cashflow_book.copy()
cashflow_book_fcst["deposit"] = 0
cashflow_book_fcst["screen_insurance"] = 0

var_term = ["term" + str(ele) for ele in range(1,12)]


In [74]:
cashflow_fcst

,date,netflow_old,inflow_deposit,inflow_screen_ins,inflow_payment,outflow_phone_cost,outflow_traffic_fee,outflow_mayi_fee,outflow_insurance,netflow_new,orders_new,cash_on_book
0,2024-01-29,33063.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,333063.05
1,2024-01-30,32151.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-31,8664.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-02-01,18469.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-02-02,11067.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
696,2025-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
697,2025-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698,2025-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
699,2025-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# let the loop begin

In [75]:
%%time

for i in range(1, cashflow_fcst.shape[0]):
    date_i = cashflow_fcst.iloc[i]["date"]

    cash_to_use_i = np.maximum(cashflow_fcst.iloc[i-1]["cash_on_book"] - cash_balance_floor, 0)  
    
    if i == 1:
        matrix_cash_left_i   =  device_matrix["cash_cutoff_cumu"].max()
        delayed_orders_i = pd.DataFrame()

    if (cash_to_use_i > 0) & (date_i < as_of_dt + relativedelta(months = reinvest_petiod)):
        orders_i, matrix_cash_left_i, cash_on_hand_left_i = order_rundown(cash_on_hand = cash_to_use_i,
                                                            matrix_cash_left = matrix_cash_left_i,
                                                            matrix = device_matrix)

        alloc_orders_i, delayed_orders_i = order_allocate(new_orders = orders_i,
                                                          delayed_orders = delayed_orders_i,
                                                          daily_cap = daily_volume_cap)

        alloc_orders_i["payment_date"] = cashflow_fcst.iloc[i]["date"]
        alloc_orders_i["deposit"]  = alloc_orders_i["deposit_mean"] * alloc_orders_i["num_device"] 
        alloc_orders_i["traffic_fee"]  = alloc_orders_i["official_px_mean"] * alloc_orders_i["px_multiplier_mean"] * alloc_orders_i["num_device"] * traffic_fee_ratio
        alloc_orders_i["mayi_fee"]  = alloc_orders_i["num_device"] * mayi_fee
        alloc_orders_i["insurance_fee"]  = alloc_orders_i["pur_px_mean"] * alloc_orders_i["num_device"] * insurance_fee_ratio
        alloc_orders_i["screen_insurance"]  = alloc_orders_i["screen_ins_mean"] * alloc_orders_i["num_device"]
        alloc_orders_i["term1"] = np.where(alloc_orders_i["onedollar"] == 0,
                                           alloc_orders_i["monthly_pay_mean"] * alloc_orders_i["num_device"],
                                           1 * alloc_orders_i["num_device"])
        for j in range(2,12):
            alloc_orders_i["term"+str(j)] = alloc_orders_i["monthly_pay_mean"] * alloc_orders_i["num_device"]

        alloc_orders_i["term12"] = (alloc_orders_i["official_px_mean"] * alloc_orders_i["px_multiplier_mean"] - alloc_orders_i["deposit_mean"]) * alloc_orders_i["num_device"] - \
                                   alloc_orders_i.loc[:, "term1":"term11"].sum(axis = 1)

        cashflow_book_fcst_i = alloc_orders_i.melt(id_vars=["device","onedollar","lease_term","deposit_mean","pur_px_mean","screen_ins_mean",
                                                  "official_px_mean","monthly_pay_mean","px_multiplier_mean" ,"num_device",
                                                 "cost","payment_date","deposit","traffic_fee","mayi_fee","insurance_fee","screen_insurance"])
        cashflow_book_fcst_i["payment_date_new"] = cashflow_book_fcst_i["payment_date"].dt.date + (cashflow_book_fcst_i["variable"].str[4:].astype(int)-1).apply(lambda m: pd.DateOffset(months=m))
        cashflow_book_fcst_i["payment_date_new"] = pd.to_datetime(cashflow_book_fcst_i["payment_date_new"]).dt.normalize()
        cashflow_book_fcst_i["final_payment_date"] = cashflow_book_fcst_i["payment_date"].dt.date + (cashflow_book_fcst_i["lease_term"]-1).apply(lambda m: pd.DateOffset(months=m))
        cashflow_book_fcst_i["final_payment_date"] = pd.to_datetime(cashflow_book_fcst_i["final_payment_date"]).dt.normalize()
        for column_i in ["cost", "deposit", "traffic_fee", "mayi_fee", "insurance_fee", "screen_insurance"]:
            cashflow_book_fcst_i.loc[cashflow_book_fcst_i["variable"] != "term1", column_i] = 0
        cashflow_book_fcst = pd.concat([cashflow_book_fcst, cashflow_book_fcst_i])
        cashflow_fcst.at[i, "orders_new"] = alloc_orders_i["num_device"].sum()
    elif cash_to_use_i <= 0:
        print(date_i, cash_to_use_i)
        

    cashflow_book_agg_i = cashflow_book_fcst[cashflow_book_fcst["payment_date_new"] == date_i].groupby("payment_date_new", as_index = False) \
                                  .agg(inflow_deposit = ("deposit", "sum"),
                                       inflow_screen_ins = ("screen_insurance", "sum"),
                                       inflow_payment = ("value", "sum"),
                                       outflow_phone_cost = ("cost", "sum"),
                                       outflow_traffic_fee = ("traffic_fee", "sum"),
                                       outflow_mayi_fee = ("mayi_fee", "sum"),
                                       outflow_insurance = ("insurance_fee", "sum"))
    cashflow_book_agg_i["netflow_new"] = cashflow_book_agg_i["inflow_deposit"] + cashflow_book_agg_i["inflow_screen_ins"] + cashflow_book_agg_i["inflow_payment"] - \
                                     cashflow_book_agg_i["outflow_phone_cost"] - cashflow_book_agg_i["outflow_traffic_fee"] - cashflow_book_agg_i["outflow_mayi_fee"] - \
                                     cashflow_book_agg_i["outflow_insurance"]

    for column_i in ["inflow_deposit", "inflow_screen_ins", "inflow_payment", 
                     "outflow_phone_cost", "outflow_traffic_fee", "outflow_mayi_fee", 
                     "outflow_insurance", "netflow_new"]:
        cashflow_fcst.at[i,column_i] = 0 if len(cashflow_book_agg_i.index) == 0 else cashflow_book_agg_i.loc[0, column_i]
     
    cashflow_fcst.at[i, "cash_on_book"] = cashflow_fcst.at[i-1, "cash_on_book"] + cashflow_fcst.at[i, "netflow_new"] 



CPU times: total: 13 s
Wall time: 13 s


In [76]:
cashflow_fcst.tail()

,date,netflow_old,inflow_deposit,inflow_screen_ins,inflow_payment,outflow_phone_cost,outflow_traffic_fee,outflow_mayi_fee,outflow_insurance,netflow_new,orders_new,cash_on_book
696,2025-12-25,NaN,0,0,-268.696598,0.0,0,0,0,-268.696598,NaN,15326240.008071
697,2025-12-26,NaN,0,0,19928.474442,0.0,0,0,0,19928.474442,NaN,15346168.482513
698,2025-12-27,NaN,0,0,15978.751624,0.0,0,0,0,15978.751624,NaN,15362147.234137
699,2025-12-28,NaN,0,0,17976.095577,0.0,0,0,0,17976.095577,NaN,15380123.329714
700,2025-12-29,NaN,0,0,0,0,0,0,0,0,NaN,15380123.329714


In [77]:
as_of_dt = dt.datetime(2024, 1, 29)

initial_capital = 6500000
cash_balance = 300000
cash_balance_floor = 50000
end_cash_balance = cashflow_fcst["cash_on_book"].iloc[-1] * (1-withdraw_fee_ratio)
end_date = cashflow_fcst["date"].iloc[-1]
end_return = end_cash_balance / initial_capital - 1
end_date, end_return


(Timestamp('2025-12-29 00:00:00'), 1.3661728199559402)